In [9]:
import os
from datetime import datetime
import random

import numpy as np
from math import *

import tensorflow as tf

import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import KFold

# Tensorflow

In [10]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
tf.test.is_built_with_cuda()

True

# Case

In [11]:
folders = ['Rough','UTS','Elon']
index = 0
CASE = folders[index]

In [12]:
from prepross import importForK
x,y = importForK(index)

# Network

In [13]:
# Hyperparameters
lossfn = [tf.keras.losses.MeanSquaredError(),tf.keras.losses.MeanAbsolutePercentageError(),tf.keras.losses.MeanAbsoluteError()]
lossfn = lossfn[index]
optimizer = [tf.keras.optimizers.Adam(learning_rate=i,beta_1=0.09,beta_2=0.9) for i in [5e-2,1e-2,5e-3]]
optimizer = optimizer[index]
metrics = ['mean_squared_error','mean_absolute_error',tf.keras.losses.MeanAbsolutePercentageError()]

In [14]:
#Give a random seed
tf.random.set_seed(random.randint(1,1000))
from networks import create_model,compile_and_fit

# K-Fold cross validation

In [15]:
def kfoldTrain(xd,yd,lossfn,mArch,n_split,modelName):
  error = {}
  error['MAE'] = 0
  error["MSE"] = 0
  error['MRE'] = 0
  i=1
  for train_index,test_index in KFold(n_split).split(xd):
    x_train,x_test=xd.values[train_index],xd.values[test_index]
    y_train,y_test=yd.values[train_index],yd.values[test_index]
    
    model = create_model(mArch,modelName+"_v"+str(i),len(x.columns))
    # Print Model Summary
    model.summary()

    # Compile And Fit
    history = compile_and_fit(CASE,model,optimizer,lossfn,metrics,
                            modelName+'_v'+str(i),x_train,y_train,
                            verbose = 0,batch_size=3,MAX_EPOCHS=120,
                            patience = 50,record = True)
 
    error['MSE'] += model.evaluate(x_test,y_test)[0]
    error['MAE'] += model.evaluate(x_test,y_test)[1]
    error['MRE'] += model.evaluate(x_test,y_test)[2]
    i+=1
  return error

In [16]:
performance = {}
# Create a basic model Architecture
mArch = [[4,'tanh',1],[5,'linear',1]]

modelArchitecture = [mArch[0],mArch[1],[1,'linear',1]]
print(modelArchitecture)
modelName = ''
for u,fn,b in modelArchitecture:
    modelName += str(u)+fn+str(b)+'_'

performance[modelName] = kfoldTrain(x,y,lossfn,modelArchitecture,10,modelName)
# print(val_performance[modelName])
# print(performance[modelName])

[[4, 'tanh', 1], [5, 'linear', 1], [1, 'linear', 1]]
Model: "4tanh1_5linear1_1linear1__v1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 4)                 32        
                                                                 
 dense_31 (Dense)            (None, 5)                 25        
                                                                 
 dense_32 (Dense)            (None, 1)                 6         
                                                                 
Total params: 63
Trainable params: 63
Non-trainable params: 0
_________________________________________________________________
1/1 [==============================] - 0s 20ms/step - loss: 16532.2988 - mean_squared_error: 16532.2988 - mean_absolute_error: 126.0070 - mean_absolute_percentage_error: 490.6319
Model: "4tanh1_5linear1_1linear1__v2"
____________________________________